In [1414]:
import cv2
import numpy as np
from random import randint

In [1415]:
def resize(img, times):
    new_width = int(img.shape[1] * times)
    new_height = int(img.shape[0] * times)
    return cv2.resize( img, (new_width, new_height), cv2.INTER_LANCZOS4)

In [1416]:
def attach_background(img, background):

    bg_height,  bg_width,  _ = background.shape
    img_height, img_width, _ = img.shape
    
    assert( bg_height > img_height)
    assert( bg_width > img_width)
    
    offset_var_x = (bg_width - img_width)   // 4
    offset_var_y = (bg_height - img_height) // 4
    x_offset = (bg_width - img_width)   // 2 + randint(-offset_var_x, offset_var_x)
    y_offset = (bg_height - img_height) // 2 + randint(-offset_var_y, offset_var_y)
    res = background.copy()
    res[y_offset:y_offset+img_height, x_offset:x_offset+img_width] = img

    return res

In [1417]:
def add_noise(img):
    mean = np.random.normal(0.25, 0.05)
    var  = np.abs(np.random.normal(0.075, 0.05))
    gauss_noise = (np.random.normal(mean, var, img.shape) * 255).astype("uint8")
    return cv2.add(img, gauss_noise)

In [1418]:
def random_line(img):
    position = randint(0, img.shape[1])
    width = randint(2, 4)
    res = cv2.line(img, (position,0), (position,img.shape[0]), (50, 50, 50), width) 
    return res

In [1419]:
a = np.array([1, 2])
b = np.array([3, 4])
print(a * b)

[3 8]


In [1420]:
def proj( point, eye):
    delta = point - eye
    assert(delta[2] > 1e-3)
    param = -eye[2] / delta[2]
    proj = delta * param + eye
    return proj[:2]

def perspective_transform(img):
    img_height, img_width, _ = img.shape

    vec_var = 0.15
    vec1 = np.array([0, -1, 0]) + np.random.uniform(-vec_var, vec_var, 3)
    vec2 = np.array([1, 0, 0])  + np.random.uniform(-vec_var, vec_var, 3)
    vec1 = vec1 / np.linalg.norm(vec1) * (img_height / 2)
    vec2 = vec2 / np.linalg.norm(vec2) * (img_width  / 2)
    depth = max(img_width, img_height) / 2
    new_centre = np.array([img_width  / 2, img_height / 2, depth // 2])
    left_up    = new_centre + vec1 - vec2
    left_down  = new_centre - vec1 - vec2
    right_up   = new_centre + vec1 + vec2
    right_down = new_centre - vec1 + vec2

    eye_pos    = np.array([img_width  / 2, img_height / 2, -depth // 2])
    left_up    = proj(left_up, eye_pos)
    left_down  = proj(left_down, eye_pos)
    right_up   = proj(right_up, eye_pos)
    right_down = proj(right_down, eye_pos)

    pts1 = np.float32([[0, 0],          [img_width, 0],
                       [0, img_height], [img_width, img_height]])
    pts2 = np.float32([left_up,   right_up,
                       left_down, right_down])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result = cv2.warpPerspective(img, matrix, (img_width, img_height))

    left_x  = int(max(left_up[0], left_down[0]))
    right_x = int(min(right_up[0], right_down[0]))
    up_y    = int(max(left_up[1], right_up[1]))
    down_y  = int(min(left_down[1], right_down[1]))

    result = result[up_y:down_y, left_x:right_x]

    return result

In [1421]:
def change_light(img):
    img_height, img_width, _ = img.shape
    centre_x = randint( int(-img_width * 0.2), int(img_width * 1.2))
    centre_y = randint( int(-img_height * 0.2), int(img_height * 1.2))
    light_centre = np.array([centre_x, centre_y])
    max_val = 2
    speed = -0.0015
    for y in range(0, img_height):
         for x in range(0, img_width):
            dist = np.linalg.norm(np.array([x, y]) - light_centre)
            brightness = speed * dist + max_val
            res = np.clip(img[y, x] * brightness, 0, 255)
            img[y, x] = res
    return img

In [1422]:
def generate(img, background):

    img = img[10:img.shape[0]-20, 10:img.shape[1]-20]

    angle = 0 # randint(-10, 10)
    background_center = tuple(np.array(background.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(background_center, angle, 1.0)
    background = cv2.warpAffine(background, rot_mat, background.shape[1::-1], flags=cv2.INTER_LINEAR)

    bg_height,  bg_width,  _ = background.shape
    img_height, img_width, _ = img.shape

    if img_height > img_width and bg_height < bg_width or \
       img_height < img_width and bg_height > bg_width:
        background = cv2.rotate(background, cv2.ROTATE_90_CLOCKWISE)
        bg_height,  bg_width,  _ = background.shape

    best_proportion = 1.2

    x_prop = bg_width / img_width
    y_prop = bg_height / img_height
    min_prop = min(x_prop, y_prop)
    bg_scale = best_proportion / min_prop
    
    background = resize(background, bg_scale)

    blur_prob = randint(0, 100)
    if blur_prob < 40:
        img = add_noise(img)
    elif blur_prob < 80:
        kernel_sz = randint(7, 13)
        if kernel_sz % 2 == 0:
            kernel_sz += 1
        img = cv2.GaussianBlur(img, (kernel_sz, kernel_sz), cv2.BORDER_TRANSPARENT)

    if randint(0, 100) < 30:
        img = random_line(img)

    res = attach_background(img, background)
    res = cv2.GaussianBlur(res, (7, 7), cv2.BORDER_TRANSPARENT)
    res = perspective_transform(res)
    
    if randint(0, 100) < 40:
        res = change_light(res)
    
    return res

In [1423]:
def generate_loop():
    mul_num = 10
    res_num = 91
    for img_id in range(10, 11):
        print(res_num)
        for _ in range(mul_num):
            img = cv2.imread(f'./docs/{img_id}.png')
            bg_id = randint(1, 10)
            background = cv2.imread(f'./background/{bg_id}.jpg') 
            res = generate(img, background)
            cv2.imwrite(f'./generated_photos/{res_num}.png', res)
            res_num += 1

In [1424]:
generate_loop()

91
